# Make download list
#### Read in csv catalog and generate text file to use to call download_from_MAST on UChicago Midway cluster

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import alderaan.io as io

In [2]:
MAINPATH = '/Users/research/projects/alderaan/'
CSV_FILE = MAINPATH + 'Catalogs/simulated_catalog_eccentric.csv'

LISTNAME = "-sim-BIG-eccentric"

# Read in Catalog data

In [3]:
# Read in the data from csv file
print('Reading in data from csv file')

# read in a csv file containing info on targets
csv_keys, csv_values = io.read_csv_file(CSV_FILE)

# put these csv data into a dictionary
target_dict = {}
for k in csv_keys: 
    target_dict[k] = io.get_csv_data(k, csv_keys, csv_values)
    
    
for k in target_dict.keys():
    target_dict[k] = np.array(target_dict[k])

Reading in data from csv file


# Pull desired KOIs

In [4]:
npl = np.array(target_dict["npl"], dtype="int")
rp  = np.array(target_dict["prad"], dtype="float")

use = (npl == 1)*(rp > 4.0)

koi_list = np.random.choice(target_dict["koi_id"][use], np.sum(use), replace=False)

In [5]:
downloadfile = MAINPATH + "Temp/" + "download_from_MAST" + LISTNAME + ".sh"

with open(downloadfile, "w") as outfile:
    outfile.write("module unload python\n")
    outfile.write("module load python/cpython-3.7.0\n\n")

    
    for i, koi in enumerate(koi_list):
        outstring = "python3 download_from_MAST.py --mission='Kepler' --target='" + koi
        outstring += "' --primary_dir='/home/gjgilbert/projects/alderaan/'\n"
        
        outfile.write(outstring)

In [6]:
wrapperfile = MAINPATH + "Temp/" + "download_wrapper" + LISTNAME + ".wrap"

with open(wrapperfile, "w") as outfile:
    for i, koi in enumerate(koi_list):
        outstring = "%run download_from_MAST.py --mission='Kepler' --target='" + koi
        outstring += "' --primary_dir='/Users/research/projects/alderaan/'\n"
        
        outfile.write(outstring)

In [7]:
targetfile = MAINPATH + "Temp/" + "target_list" + LISTNAME + ".txt"

with open(targetfile, "w") as outfile:
    for i, koi in enumerate(koi_list):
        sim_id = "S" + koi[1:]
        
        outfile.write(sim_id + "\n")